In [15]:
# (1) 데이터로드
import pandas as pd
import numpy as np
from keras.utils import np_utils

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기

# gcc6_0~3
gcc_version = 'gcc5'
gcc6_0_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_0_32.csv", index_col=0)
gcc6_1_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_1_32.csv", index_col=0)
gcc6_2_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_2_32.csv", index_col=0)
gcc6_3_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_3_32.csv", index_col=0)

# 형태 출력
print(gcc6_0_32.shape, gcc6_1_32.shape, gcc6_2_32.shape, gcc6_3_32.shape) # train

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_0_32.reset_index(inplace=True, drop=True)
gcc6_1_32.reset_index(inplace=True, drop=True)
gcc6_2_32.reset_index(inplace=True, drop=True)
gcc6_3_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
gcc6_0_32.head()

(2686650, 2) (1878528, 2) (1957461, 2) (2416238, 2)
reset_index 완료


,bin,label
0,55,1
1,48,0
2,89,0
3,e5,0
4,48,0


In [16]:
# (2-1) 데이터체크 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_0_32['bin'].unique()),len(gcc6_1_32['bin'].unique()))
print(len(gcc6_2_32['bin'].unique()),len(gcc6_3_32['bin'].unique()))

256 256
256 256


In [17]:
# (3) get_dummies 변환 

# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_0_32_onehot = pd.get_dummies(gcc6_0_32)
gcc6_1_32_onehot = pd.get_dummies(gcc6_1_32)
gcc6_2_32_onehot = pd.get_dummies(gcc6_2_32)
gcc6_3_32_onehot = pd.get_dummies(gcc6_3_32)
print('원핫인코딩완료')

print(gcc6_0_32_onehot.shape, gcc6_1_32_onehot.shape, gcc6_2_32_onehot.shape, gcc6_3_32_onehot.shape)

원핫인코딩완료
(2686650, 257) (1878528, 257) (1957461, 257) (2416238, 257)


In [18]:
# (4) 데이터 체크 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함

# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_0_32_onehot['label'].value_counts(), '\n', gcc6_1_32_onehot['label'].value_counts())
print(gcc6_2_32_onehot['label'].value_counts(), '\n', gcc6_3_32_onehot['label'].value_counts())

0    2682208
1       4442
Name: label, dtype: int64 
 0    1875181
1       3347
Name: label, dtype: int64
0    1954180
1       3281
Name: label, dtype: int64 
 0    2413218
1       3020
Name: label, dtype: int64


In [19]:
# 전체 행렬에 대한 value_counts()
gcc6_0_32['bin'].value_counts()

00    563504
48    337916
8b    176237
ff    170570
89    115546
       ...  
93       238
b1       222
a6       206
9e       181
a9       180
Name: bin, Length: 256, dtype: int64

In [6]:
# (5-1) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx0 = gcc6_0_32_onehot[gcc6_0_32_onehot['label']==1].index  # 407, 474 ...
ls0 = list(idx0)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx0 = [] 
# left_idx, right_idx = -3, 4  # 3개씩
left_idx, right_idx = -5, 11 # 10개씩
# left_idx, right_idx = -30, 31 # 30개씩
# left_idx, right_idx = -90, 91 # 90개씩


# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx0.extend(list(idx0 + k)) # index 형이라서 가능

ls_idx0 = list(set(ls_idx0))    
# print(len(ls_idx0))

ls_idx0.sort() # 인덱스 정렬
ls_idx0 = list(filter(lambda x: x>0, ls_idx0)) # 0이상인 값들만 
ls_idx0 = list(filter(lambda x: x<len(gcc6_0_32_onehot), ls_idx0))

print('gcc6_0_32', len(ls_idx0))

# loc 로 수정필요
gcc6_0_32_onehot_30gram = gcc6_0_32_onehot.loc[ls_idx0,:].copy()

gcc6_0_32 92464


In [7]:
# (5-2) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx1 = gcc6_1_32_onehot[gcc6_1_32_onehot['label']==1].index  # 407, 474 ...
ls1 = list(idx1)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx1 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx1.extend(list(idx1 + k)) # index 형이라서 가능

ls_idx1 = list(set(ls_idx1))    
# print(len(ls_idx0))

ls_idx1.sort() # 인덱스 정렬
ls_idx1 = list(filter(lambda x: x>0, ls_idx1)) # 0이상인 값들만 
ls_idx1 = list(filter(lambda x: x<len(gcc6_1_32_onehot), ls_idx1))

print('gcc6_1_32', len(ls_idx1))

# loc 로 수정필요
gcc6_1_32_onehot_30gram = gcc6_1_32_onehot.loc[ls_idx1,:].copy()


gcc6_1_32 66823


In [8]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx2 = gcc6_2_32_onehot[gcc6_2_32_onehot['label']==1].index  # 407, 474 ...
ls2 = list(idx2)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx2 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx2.extend(list(idx2 + k)) # index 형이라서 가능

ls_idx2 = list(set(ls_idx2))    
# print(len(ls_idx0))

ls_idx2.sort() # 인덱스 정렬
ls_idx2 = list(filter(lambda x: x>0, ls_idx2)) # 0이상인 값들만 
ls_idx2 = list(filter(lambda x: x<len(gcc6_2_32_onehot), ls_idx2))

print('gcc6_2_32', len(ls_idx2))

# loc 로 수정필요
gcc6_2_32_onehot_30gram = gcc6_2_32_onehot.loc[ls_idx2,:].copy()


gcc6_2_32 66460


In [9]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx3 = gcc6_3_32_onehot[gcc6_3_32_onehot['label']==1].index  # 407, 474 ...
ls3 = list(idx3)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx3 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx3.extend(list(idx3 + k)) # index 형이라서 가능

ls_idx3 = list(set(ls_idx3))    
# print(len(ls_idx0))

ls_idx3.sort() # 인덱스 정렬
ls_idx3 = list(filter(lambda x: x>0, ls_idx3)) # 0이상인 값들만 
ls_idx3 = list(filter(lambda x: x<len(gcc6_3_32_onehot), ls_idx3))

print('gcc6_3_32', len(ls_idx3))

# loc 로 수정필요
gcc6_3_32_onehot_30gram = gcc6_3_32_onehot.loc[ls_idx3,:].copy()


gcc6_3_32 61412


In [10]:
# (7) 훈련데이터, 훈련라벨 분리 012

# 훈련 데이터, 훈련 라벨
x_gcc6_0_32 = gcc6_0_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_0_32 = gcc6_0_32_onehot_30gram['label'].to_numpy()

x_gcc6_1_32 = gcc6_1_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_1_32 = gcc6_1_32_onehot_30gram['label'].to_numpy()

x_gcc6_2_32 = gcc6_2_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_2_32 = gcc6_2_32_onehot_30gram['label'].to_numpy()

# 테스트 데이터, 테스트 라벨
x_gcc6_3_32 = gcc6_3_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_3_32 = gcc6_3_32_onehot_30gram['label'].to_numpy()

# train
print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

# test
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(92464, 256) (92464,)
(66823, 256) (66823,)
(66460, 256) (66460,)
(61412, 256) (61412,)


In [11]:
# (8) 훈련데이터, 훈련라벨 reshape 작업 => # ex - (605, 1, 256)

## gcc6_0_32
x_gcc6_0_32 = x_gcc6_0_32.reshape(x_gcc6_0_32.shape[0], 1, x_gcc6_0_32.shape[1])
y_gcc6_0_32 = y_gcc6_0_32.reshape(y_gcc6_0_32.shape[0], 1, 1)

## gcc6_1_32
x_gcc6_1_32 = x_gcc6_1_32.reshape(x_gcc6_1_32.shape[0], 1, x_gcc6_1_32.shape[1])
y_gcc6_1_32 = y_gcc6_1_32.reshape(y_gcc6_1_32.shape[0], 1, 1)

## gcc6_2_32
x_gcc6_2_32 = x_gcc6_2_32.reshape(x_gcc6_2_32.shape[0], 1, x_gcc6_2_32.shape[1])
y_gcc6_2_32 = y_gcc6_2_32.reshape(y_gcc6_2_32.shape[0], 1, 1)

### test data
## gcc6_3_32
x_gcc6_3_32 = x_gcc6_3_32.reshape(x_gcc6_3_32.shape[0], 1, x_gcc6_3_32.shape[1])
y_gcc6_3_32 = y_gcc6_3_32.reshape(y_gcc6_3_32.shape[0], 1, 1)

print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

## test data
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(92464, 1, 256) (92464, 1, 1)
(66823, 1, 256) (66823, 1, 1)
(66460, 1, 256) (66460, 1, 1)
(61412, 1, 256) (61412, 1, 1)


In [12]:
# (9) 최종 훈련데이터, 훈련라벨 concatenate 작업
# 0123 / 3
x_gcc6_0123_32 = np.concatenate((x_gcc6_0_32, x_gcc6_1_32, x_gcc6_2_32, x_gcc6_3_32), axis=0)
y_gcc6_0123_32 = np.concatenate((y_gcc6_0_32, y_gcc6_1_32, y_gcc6_2_32, y_gcc6_3_32), axis=0)
print(x_gcc6_0123_32.shape, y_gcc6_0123_32.shape)

(287159, 1, 256) (287159, 1, 1)


In [13]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 교차검증 kfold
from sklearn.model_selection import KFold

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# K-Fold 객체 생성
kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전

# Confusion Matrix
from sklearn.metrics import confusion_matrix

# ROC Curve
from sklearn.metrics import roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회
for train, validation in kf.split(x_gcc6_0123_32, y_gcc6_0123_32):
    
    xInput = Input(batch_shape=(None, 1, 256)) # 
    xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
    xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

    model1 = Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    #print(model1.summary())
    print('======Training stage======')
    model1.fit(x_gcc6_0123_32[train],
               y_gcc6_0123_32[train],
               epochs = 3,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict    
    # predict 값
    k_pr = model1.predict(x_gcc6_0123_32[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_0123_32[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
======Training stage======
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/3
258443/258443 [==============================] - 25s 98us/step - loss: 0.1437 - accuracy: 0.9506
Epoch 2/3
258443/258443 [==============================] - 24s 94us/step - loss: 0.1341 - accuracy: 0.9508
Epoch 3/3
258443/258443 [==============================] - 24s 92us/step - loss: 0.1349 - accuracy: 0.9508


accuracy_score 0.9519779913636996
recall_score 0.0
precision_score 0.0
f1_score 0.0
======Training stage======
Epoch 1/3
258443/258443 [==============================] - 24s 94us/step - loss: 0.1442 - accuracy: 0.9507
Epoch 2/3
258443/258443 [==============================] - 26s 99us/step - loss: 0.1341 - accuracy: 0.9509
Epoch 3/3
258443/258443 [==============================] - 23s 90us/step - loss: 0.1349 - accuracy: 0.9509


accuracy_score 0.9517342248223987
recall_score 0.0
precision_score 0.0
f1_score 0.0
======Training stage======
Epoch 1/3
258443/258443 [==============================] - 26s 101us/step - loss: 0.1442 - accuracy: 0.9507
Epoch 2/3
258443/258443 [==============================] - 27s 104us/step - loss: 0.1339 - accuracy: 0.9508
Epoch 3/3
258443/258443 [==============================] - 27s 103us/step - loss: 0.1340 - accuracy: 0.9508


accuracy_score 0.9521521103217718
recall_score 0.0
precision_score 0.0
f1_score 0.0
======Training stage======
Epoch 1/3
258443/258443 [==============================] - 27s 103us/step - loss: 0.1354 - accuracy: 0.9508
Epoch 2/3
258443/258443 [==============================] - 26s 99us/step - loss: 0.1257 - accuracy: 0.9509
Epoch 3/3
258443/258443 [==============================] - 26s 100us/step - loss: 0.1266 - accuracy: 0.9507


accuracy_score 0.9504805683242792
recall_score 0.0
precision_score 0.0
f1_score 0.0
======Training stage======
Epoch 1/3
258443/258443 [==============================] - 26s 99us/step - loss: 0.1327 - accuracy: 0.9516
Epoch 2/3
258443/258443 [==============================] - 25s 97us/step - loss: 0.1232 - accuracy: 0.9522
Epoch 3/3
258443/258443 [==============================] - 25s 98us/step - loss: 0.1239 - accuracy: 0.9525


...

accuracy_score 0.9500278590332916
recall_score 0.0
precision_score 0.0
f1_score 0.0
======Training stage======
Epoch 1/3
258443/258443 [==============================] - 28s 107us/step - loss: 0.1357 - accuracy: 0.9506
Epoch 2/3
258443/258443 [==============================] - 31s 120us/step - loss: 0.1251 - accuracy: 0.9512
Epoch 3/3
258443/258443 [==============================] - 29s 111us/step - loss: 0.1255 - accuracy: 0.9515


accuracy_score 0.9506198634907369
recall_score 0.0
precision_score 0.0
f1_score 0.0
======Training stage======
Epoch 1/3
258443/258443 [==============================] - 30s 116us/step - loss: 0.1351 - accuracy: 0.9515
Epoch 2/3
258443/258443 [==============================] - 28s 109us/step - loss: 0.1251 - accuracy: 0.9515
Epoch 3/3
258443/258443 [==============================] - 28s 108us/step - loss: 0.1248 - accuracy: 0.9520


accuracy_score 0.9271834517342248
recall_score 0.0862190812720848
precision_score 0.13260869565217392
f1_score 0.10449678800856532
======Training stage======
Epoch 1/3
258443/258443 [==============================] - 28s 110us/step - loss: 0.1350 - accuracy: 0.9519
Epoch 2/3
258443/258443 [==============================] - 30s 115us/step - loss: 0.1251 - accuracy: 0.9517
Epoch 3/3
258443/258443 [==============================] - 25s 96us/step - loss: 0.1257 - accuracy: 0.9520


accuracy_score 0.9508984538236523
recall_score 0.0
precision_score 0.0
f1_score 0.0
======Training stage======
Epoch 1/3
258444/258444 [==============================] - 25s 97us/step - loss: 0.1353 - accuracy: 0.9517
Epoch 2/3
258444/258444 [==============================] - 23s 88us/step - loss: 0.1249 - accuracy: 0.9526
Epoch 3/3
258444/258444 [==============================] - 22s 84us/step - loss: 0.1247 - accuracy: 0.9525


accuracy_score 0.9236984154623019
recall_score 0.0766501064584812
precision_score 0.10821643286573146
f1_score 0.08973826339842128

K-fold cross validation Accuracy: [0.9519779913636996, 0.9517342248223987, 0.9521521103217718, 0.9504805683242792, 0.9246064911547569, 0.9500278590332916, 0.9506198634907369, 0.9271834517342248, 0.9508984538236523, 0.9236984154623019]

K-fold cross validation Recall: [0.0, 0.0, 0.0, 0.0, 0.10083449235048679, 0.0, 0.0, 0.0862190812720848, 0.0, 0.0766501064584812]

K-fold cross validation Precision: [0.0, 0.0, 0.0, 0.0, 0.14257620452310718, 0.0, 0.0, 0.13260869565217392, 0.0, 0.10821643286573146]

K-fold cross validation F1-Score: [0.0, 0.0, 0.0, 0.0, 0.11812627291242363, 0.0, 0.0, 0.10449678800856532, 0.0, 0.08973826339842128]


In [14]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9433379429531115
10-Fold Cross_validation. Recall : 0.02637036800810528
10-Fold Cross_validation. Precision : 0.038340133304101257
10-Fold Cross_validation. F1-Score : 0.03123613243194102
